In [2]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen

import random
import numpy as np
from tqdm import tqdm_notebook as tqdm
#from tqdm import tqdm.notebook.tqdm as tqdm
from collections import defaultdict

import warnings
warnings.filterwarnings("ignore")

## Se importan los dataframes 

In [3]:
# load all metadata
#hau nire diskan daukat SAD karpetan 2020-2021

video_games = []#
insurances = []   
#dfMergedfMeta= pd.read_csv('Cell_Phones/SamsungAppleXiaomiReviews_prueba.csv')
video_games= pd.read_csv('VG-Reviews5AndMetaElecNintSonyMic_v2.csv')
insurances = pd.read_csv('HRBlockIntuitReviewsTrainDev_vLast7.csv', low_memory=False)


In [4]:
video_games.keys()

Index(['Unnamed: 0', 'asin', 'vote', 'style', 'feature', 'similar_item',
       'brand', 'also_buy', 'category', 'title', 'main_cat', 'description',
       'reviewTime', 'overall', 'reviewerID', 'reviewerName', 'unixReviewTime',
       'reviewText', 'summary', 'City', 'State', 'Country', 'Postal Code'],
      dtype='object')

In [5]:
video_games['brand'].value_counts()

by\n    \n    Electronic Arts                        19949
by\n    \n    Nintendo                               16812
by\n    \n    Sony                                   10915
by\n    \n    Microsoft                               8810
Nintendo                                              4660
Electronic Arts                                       1780
Sony                                                  1697
Microsoft                                             1230
Sony Computer Entertainment                            438
by\n    \n    Sony Computer Entertainment America      106
Nintendo of America                                     98
Electronic Arts, Inc.                                   78
by\n    \n    Sony Computer Entertainment               74
Sony Entertainment                                      52
Microsoft Corporation                                   44
Super Nintendo Super Castlevania IV                     22
Electronc Arts                                          

In [6]:
insurances.keys()

Index(['Unnamed: 0', 'asin', 'brand', 'price', 'title', 'Unnamed: 5',
       'category', 'rank', 'description', 'main_cat', 'details', 'overall',
       'vote_y', 'verified_y', 'reviewTime_y', 'reviewerID_y',
       'reviewerName_y', 'reviewText', 'summary', 'unixReviewTime', 'State',
       'Country', 'Postal Code'],
      dtype='object')

In [7]:
insurances['brand'].value_counts()

Intuit                              72752
H&R Block                           16242
by\n    \n    Intuit                 9488
HRBB9                                 657
H&amp;R Block                         358
by\n    \n    H&R Block               195
Intuit, Inc.                           27
H&R BLCOK                              17
John Truby Blockbuster                 15
Video Blocks                            4
Administaff HRTools                     4
H & R Block                             4
Intuit Inc.                             3
Teneron/Block Financial Software        3
H&R                                     3
H&R BLOCK                               2
Intuit Inc./BlueHippo                   2
Name: brand, dtype: int64

## Se obtienen en dataframes aparte las distintas marcas

In [8]:
dfNintendo = video_games[video_games['brand'].str.contains('Nintendo',na=False)]

In [9]:
dfSony = video_games[video_games['brand'].str.contains('Sony|PlayStation|Electr',na=False)]

In [10]:
dfMicrosoft = video_games[video_games['brand'].str.contains('[M|m]icrosoft',na=False)]

In [11]:
dfIntuit = insurances[insurances['brand'].str.contains('Intuit')]

In [12]:
dfHR = insurances[~insurances['brand'].str.contains('Intuit')]

## Se separan las reviews negativas y positivas de cada dataframe

In [13]:
dfNintendoPositives=dfNintendo[dfNintendo['overall']>3]
dfNintendoNegatives=dfNintendo[dfNintendo['overall']<=3]

In [14]:
dfSonyPositives=dfSony[dfSony['overall']>3]
dfSonyNegatives=dfSony[dfSony['overall']<=3]

In [15]:
dfMicrosoftPositives=dfMicrosoft[dfMicrosoft['overall']>3]
dfMicrosoftNegatives=dfMicrosoft[dfMicrosoft['overall']<=3]

In [16]:
dfIntuitPositives = dfIntuit[dfIntuit['overall']>3]
dfIntuitNegatives = dfIntuit[dfIntuit['overall']<=3]

In [17]:
dfHRPositives = dfHR[dfHR['overall']>3]
dfHRNegatives = dfHR[dfHR['overall']<=3]

## Se obtienen las text reviews

In [18]:
nintendoPositivesDocuments = dfNintendoPositives[dfNintendoPositives['reviewText'].notna()]['reviewText'].tolist()
nintendoNegativesDocuments = dfNintendoNegatives[dfNintendoNegatives['reviewText'].notna()]['reviewText'].tolist()

In [19]:
sonyPositivesDocuments = dfSonyPositives[dfSonyPositives['reviewText'].notna()]['reviewText'].tolist()
sonyNegativesDocuments = dfSonyNegatives[dfSonyNegatives['reviewText'].notna()]['reviewText'].tolist()

In [20]:
microsoftPositivesDocuments = dfMicrosoftPositives[dfMicrosoftPositives['reviewText'].notna()]['reviewText'].tolist()
microsoftNegativesDocuments = dfMicrosoftNegatives[dfMicrosoftNegatives['reviewText'].notna()]['reviewText'].tolist()

In [21]:
intuitPositivesDocuments = dfIntuitPositives[dfIntuitPositives['reviewText'].notna()]['reviewText'].tolist()
intuitNegativesDocuments = dfIntuitNegatives[dfIntuitNegatives['reviewText'].notna()]['reviewText'].tolist()

In [22]:
HRPositivesDocuments = dfHRPositives[dfHRPositives['reviewText'].notna()]['reviewText'].tolist()
HRNegativesDocuments = dfHRNegatives[dfHRNegatives['reviewText'].notna()]['reviewText'].tolist()

## LDA

In [47]:
def display_topics(H, W, feature_names, documents,output_name, no_top_words, no_top_documents):
    f = open(output_name + ".txt", "a")
    for topic_idx, topic in enumerate(H):
        s = "Topic: " + str(topic_idx) + " -->"
        print(s)
        f.write(s)
        s = ''.join([' ' +feature_names[i] + ' ' + str(round(topic[i], 5)) 
                for i in topic.argsort()[:-no_top_words - 1:-1]])
        print(s)
        f.write(s)
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        docProbArray=np.argsort(W[:,topic_idx])
        print(docProbArray)
    
        #for doc_index in top_doc_indices:
        #    print(documents[doc_index])
        f.write("\n")



In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

### LDANintPos_31_1.9

In [25]:
no_topics = 31

alpha = 1.9
beta = 1.9

documents = nintendoPositivesDocuments

In [26]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

In [27]:
lda_model = LatentDirichletAllocation(n_components=no_topics, doc_topic_prior=alpha, topic_word_prior=beta, max_iter=100, learning_method='online', learning_offset=50.,random_state=0).fit(tf)
lda_W = lda_model.transform(tf)
lda_H = lda_model.components_

In [49]:
display_topics(lda_H, lda_W, tf_feature_names, documents, 'LDANintPos_31_1.9')
#no_top_words = 30 
#no_top_documents = 10

Topic: 0 -->
 love 56.5796 great 30.79531 bought 28.47523 fun 24.8619 loves 21.15262 buy 20.94318 recommend 20.84963 son 20.2731 kids 18.77345 memory 16.54567 card 16.51072 got 15.37755 awesome 14.02213 family 13.70027 works 13.10907 loved 12.53892 price 11.88835 christmas 11.54587 highly 10.93933 dont 10.87158 old 10.84657 playing 10.622 best 10.42998 happy 10.42591 enjoy 9.97712 daughter 9.89878 came 9.7173 good 9.65842 play 9.54708 condition 9.45347
[ 2807  2835 14844 ... 16027 11561 12307]
Topic: 1 -->
 love 67.17905 great 32.10282 bought 29.83885 fun 25.06459 buy 22.49899 loves 22.15924 recommend 20.87419 son 20.67758 kids 18.92196 memory 16.78041 card 16.71919 got 15.49651 awesome 14.1285 family 13.78942 works 13.12111 loved 12.6001 price 11.94939 christmas 11.62732 highly 10.95748 dont 10.92965 old 10.90139 playing 10.67741 best 10.46358 happy 10.46248 enjoy 10.0031 daughter 9.97395 came 9.73875 good 9.68373 play 9.59308 condition 9.4851
[ 2835  2807 14844 ... 11434   318 12307]

### LDANintNeg_26_1.9

In [50]:
no_topics = 26

alpha = 1.9
beta = 1.9

documents = nintendoNegativesDocuments

In [51]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

In [58]:
lda_model = LatentDirichletAllocation(n_components=no_topics, doc_topic_prior=alpha, topic_word_prior=beta, max_iter=100, learning_method='online', learning_offset=50.,random_state=0).fit(tf)
lda_W = lda_model.transform(tf)
lda_H = lda_model.components_

In [53]:
display_topics(lda_H, lda_W, tf_feature_names, documents, 'LDANintNeg_26_1.9')

Topic: 0 -->
 suck 4.08839 bought 3.76344 boring 3.67473 dont 3.60581 son 3.50499 buy 3.36098 product 3.28932 kids 3.26256 im 3.23298 ok 3.1745 theres 3.1666 bored 3.15171 waste 3.14917 controllers 3.11876 tetris 3.1149 okay 3.10425 crap 3.06327 recommend 3.0441 money 3.03261 stupid 3.03129 pikachu 3.03085 sucks 3.0265 work 3.02371 alot 3.00716 rent 3.00545 got 3.00254 price 2.98927 00 2.98423 controller 2.97691 christmas 2.95337
[ 109  105 2731 ... 3261 2803 2577]
Topic: 1 -->
 suck 4.08851 bought 3.76343 boring 3.67479 dont 3.6058 son 3.50498 buy 3.36097 product 3.2893 kids 3.26256 im 3.23299 ok 3.17452 theres 3.1666 bored 3.15171 waste 3.14917 controllers 3.11873 tetris 3.11493 okay 3.10425 crap 3.06329 recommend 3.0441 money 3.03261 stupid 3.03129 pikachu 3.03082 sucks 3.0265 work 3.02371 alot 3.00716 rent 3.00545 got 3.00254 price 2.98928 00 2.98423 controller 2.97691 christmas 2.95337
[ 109  105 2731 ... 3261 2803 2577]
Topic: 2 -->
 suck 4.08844 bought 3.76343 boring 3.67478 don

### LDASonyPos_34_1.9

In [54]:
no_topics = 34

alpha = 1.9
beta = 1.9

documents = sonyPositivesDocuments

In [55]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

In [56]:
lda_model = LatentDirichletAllocation(n_components=no_topics, doc_topic_prior=alpha, topic_word_prior=beta, max_iter=100, learning_method='online', learning_offset=50.,random_state=0).fit(tf)
lda_W = lda_model.transform(tf)
lda_H = lda_model.components_

In [57]:
display_topics(lda_H, lda_W, tf_feature_names, documents, 'LDASonyPos_34_1.9')

Topic: 0 -->
 great 34.94529 awesome 23.48661 vita 20.17051 best 17.40628 ps3 14.62213 recommend 13.84084 amazing 13.02592 controller 12.45415 graphics 12.35581 good 12.30561 son 12.03316 love 11.97301 fun 11.86006 played 11.47626 loves 10.58809 ps2 10.27792 ps 10.07985 im 9.99507 loved 9.87495 ps4 9.84485 fan 9.83943 recommended 9.53266 dont 9.47562 games 9.22179 highly 8.66557 excellent 8.37514 game 8.24809 psp 8.22084 enjoyed 8.03641 series 7.90498
[20660 25091 24979 ... 13787 21947 12341]
Topic: 1 -->
 great 37.31321 awesome 22.93975 vita 20.16819 best 17.55859 ps3 14.66355 recommend 13.732 amazing 13.0199 controller 12.45406 graphics 12.34568 good 12.26966 son 12.08823 fun 11.92939 love 11.92812 played 11.47198 loves 10.58532 ps2 10.27739 ps 10.08176 im 9.99345 loved 9.87328 ps4 9.84545 fan 9.837 recommended 9.53321 dont 9.47479 games 9.21716 highly 8.66264 excellent 8.37529 psp 8.22342 game 8.2075 enjoyed 8.03582 series 7.90558
[20660 25091 24979 ... 13787 12341 16605]
Topic: 2 -

### GensimMicroPos

### GensimMicroNeg